## Train Single Task Models

In [ ]:
import tensorflow as tf
import json , os , sys

In [ ]:
sys.path.append(os.path.abspath(os.path.join('..','data_processing')))
sys.path.append(os.path.abspath(os.path.join('..','models')))

In [ ]:
from preprocessing import train_data, validation_data, test_data

In [ ]:
from efficientnetb0 import create_efficientnetb0, create_efficientnetb3

In [ ]:
from mobileNet import create_mobileNet

In [ ]:
def train_and_save_model(model_name, model_function, dropout_rate, epochs, batch_size=32, learning_rate=0.001, patience=5):


    results_dir = '../results/Single Task'
    model_dir = os.path.join(results_dir, model_name)
    os.makedirs(model_dir, exist_ok=True)
    
    model = model_function(dropout_rate)
    # Compile the model with  learning rate
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=0.5,
        patience=patience,
        min_lr=1e-6,
        verbose=1
    )

    # Train the model 
    history = model.fit(
        train_data,
        validation_data=validation_data,
        epochs=epochs,
        callbacks=[reduce_lr_callback],
        verbose=1,
        workers=6
    )

    # Evaluate on test data
    test_loss, test_accuracy = model.evaluate(test_data)

    # Save model
    model_path = os.path.join(model_dir, f'{model_name}.h5')
    model.save(model_path)

    # Gather training info and hyperparameters
    training_info = {
        "Model": model_name,
        "epochs": epochs,
        "batch_size": batch_size,
        "initial_learning_rate": learning_rate,
        "dropout_rate": dropout_rate,
        "patience": patience,
        "train_accuracy": history.history['accuracy'],            
        "val_accuracy": history.history['val_accuracy'],          
        "train_loss": history.history['loss'],                    
        "val_loss": history.history['val_loss'],                  
        "test_accuracy": test_accuracy,                           
        "test_loss": test_loss                                    
    }

    # Save training info to a JSON file
    info_path = os.path.join(model_dir, f'{model_name}_training_info.json')
    with open(info_path, 'w') as f:
        json.dump(training_info, f, indent=4)

    print(f"Model and training info saved in: {model_dir}")
    return history, training_info

In [ ]:
train_and_save_model('EfficientNetB0_Gan', create_efficientnetb0, dropout_rate=0.2, epochs=20, batch_size=32, learning_rate=0.001, patience=5)

In [ ]:
train_and_save_model('MobileNet_Gan', create_mobileNet, dropout_rate=0.2, epochs=20, batch_size=32, learning_rate=0.001, patience=5)